# Create Mesh for Project

This notebook imports a svg file and triangulates the given shape.

Libraries used in this notebook:
* `svgpathtools` to read and process an svg file
* `triangle` to triangulate

In [ ]:
!pip install svgpathtools

In [ ]:
!pip install triangle

In [ ]:
from svgpathtools import svg2paths
import numpy as np

In [ ]:
def split_into_subpaths(path):
    """
    When a cage + shape is created in inkscape, the result is a single path.
    We want to split the path into two subpaths, one for the cage and one for the shape.
    Return a list of subpaths.
    """ 
    subpaths = []
    current = []

    for i, segment in enumerate(path):
        if i == 0 or segment.start == path[i-1].end:
            current.append(segment)
        else: 
            # when the start of a segment doesn't match the end of the previous one, we have a new subpath
            subpaths.append(current)
            current = [segment]
            
    if current:
        subpaths.append(current)
    
    return subpaths

def sample_path(segments):
    """
    Converts svg path (segments) to coordinates 
    """
    points = [(seg.start.real, seg.start.imag) for seg in segments]
    points.append((segments[-1].end.real, segments[-1].end.imag))
    
    return np.array(points)

In [ ]:
paths, _ = svg2paths("data/star.svg")
path = paths[0]

subpaths = split_into_subpaths(path)
all_shapes = [sample_path(sub) for sub in subpaths]

for i, shape in enumerate(all_shapes):
    print(f"Shape {i} has {len(shape)} points")
    # print(shape)

In [ ]:
import matplotlib.pyplot as plt

# Plot the vertices to see if they look correct

# the vertices are flipped upside down, flip the y-coords before plotting
square_flipped = all_shapes[0]
star_flipped = all_shapes[1]

square_vertices = square_flipped.copy()
star_vertices = star_flipped.copy()

square_vertices[:, 1] *= -1
star_vertices[:, 1] *= -1

plt.figure(figsize=(6, 6))
plt.plot(square_vertices[:, 0], square_vertices[:, 1], marker='o')
plt.plot(star_vertices[:, 0], star_vertices[:, 1], marker='o')
plt.show()

In [ ]:
import numpy as np
import triangle as tr
import matplotlib.pyplot as plt

# In order to triangulate, we first need to define edges.
square_vertices = square_vertices[:-1]
star_vertices = star_vertices[:-1]

n_square = len(square_vertices)
n_star = len(star_vertices)

square_edges = [[i, i + 1] for i in range(n_square - 1)] + [[n_square - 1, 0]]
star_edges = [[n_square + i, n_square + i + 1] for i in range(n_star - 1)] + [[n_square + n_star - 1, n_square]]

edges = square_edges + star_edges
vertices = np.vstack([square_vertices, star_vertices])

In [ ]:
import meshplot as mp

triangulation_input = {
    'vertices': vertices,
    'segments': edges,
}

t = tr.triangulate(triangulation_input, 'pa50')

mesh_vertices = t['vertices']
mesh_triangles = t['triangles']

plot = mp.plot(mesh_vertices, mesh_triangles, shading={"wireframe": True})

In [ ]:
def save_off(filename, vertices, triangles):
    """
    Save mesh as an .off file.
    """
    with open(filename, 'w') as f:
        f.write("OFF\n")
        f.write(f"{len(vertices)} {len(triangles)} 0\n")

        for v in vertices:
            f.write(f"{v[0]} {v[1]} 0.0\n")

        for tri in triangles:
            f.write(f"3 {tri[0]} {tri[1]} {tri[2]}\n")

In [ ]:
save_off("./data/star.off", mesh_vertices, mesh_triangles)

In [ ]:
# We originally had four cage vertices (one at each corner of the square),
# but meshing added more vertices along the cage boundaries.
# Let's find out which points are now in the cage boundaries.

from collections import defaultdict

EPS = 1e-8

square_x1, square_x2 = square_vertices[0][0], square_vertices[2][0]
square_y1, square_y2 = square_vertices[0][1], square_vertices[2][1]

def is_close(a, b, eps=EPS):
    return np.abs(a - b) < eps

def find_cage_side_indices(vertices, x1, x2, y1, y2):
    """
    returns a dictionary of cage indicies, classified by left/right/bottom/top sides of the square
    """
    sides = defaultdict(list)

    for i in range(vertices.shape[0]):
        if is_close(vertices[i, 0], x1):
            sides['left'].append(i)
            
        if is_close(vertices[i, 0], x2):
            sides['right'].append(i)
            
        if is_close(vertices[i, 1], y1):
            sides['top'].append(i)
            
        if is_close(vertices[i, 1], y2):
            sides['bottom'].append(i)
    
    return sides

cage_sides = find_cage_side_indices(mesh_vertices, square_x1, square_x2, square_y1, square_y2)

for key in cage_sides:
    print(f"{key}: {cage_sides[key]}")

In [ ]:
# Sort vertices on each side (counterclock-wise)
# Useful later for displaying the cage

left_sorted = sorted(cage_sides['left'], key=lambda i: -mesh_vertices[i, 1])
right_sorted = sorted(cage_sides['right'], key=lambda i: mesh_vertices[i, 1])
bottom_sorted  = sorted(cage_sides['bottom'], key=lambda i: mesh_vertices[i, 0])
top_sorted = sorted(cage_sides['top'], key=lambda i: -mesh_vertices[i, 0])

print(f"left: {left_sorted}")
print(f"right: {right_sorted}")
print(f"bottom: {bottom_sorted}")
print(f"top: {top_sorted}")

In [ ]:
# Right now, there are too many cage vertices, which is unnecessary and might restrict the influence of individual cage vertices.
# So let's reduce the number of cage vertices.

left_reduced = [left_sorted[0]] + left_sorted[2:-1:2] + [left_sorted[-1]]
right_reduced = [right_sorted[0]] + right_sorted[2:-1:2] + [right_sorted[-1]]
bottom_reduced = [bottom_sorted[0]] + bottom_sorted[2:-1:2] + [bottom_sorted[-1]]
top_reduced = [top_sorted[0]] + top_sorted[2:-1:2] + [top_sorted[-1]]

print(f"left: {left_reduced}")
print(f"right: {right_reduced}")
print(f"bottom: {bottom_reduced}")
print(f"top: {top_reduced}")

cage_sides['left'] = left_reduced
cage_sides['right'] = right_reduced
cage_sides['bottom'] = bottom_reduced
cage_sides['top'] = top_reduced

In [ ]:
cage_indices = left_reduced[:-1] + bottom_reduced[:-1] + right_reduced[:-1] + top_reduced[:-1]

# plot the cage vertices to see if they look correct
p = mp.plot(mesh_vertices, mesh_triangles, shading={"wireframe": True})
p.add_points(mesh_vertices[cage_indices], shading={"point_size": 15, "point_color":"red"})

In [ ]:
import json

"""
In the .off file, the first four vertices correspond to the four corners of the square,
and the next 10 vertices correspond to the vertices of the star.
"""

data = {
    "square_n": n_square,
    "cage_n": len(cage_indices),
    "cage_indices": [int(i) for i in cage_indices],
    "cage_sides": cage_sides,
    "star_n": n_star,
    "star_indices": [i+n_square for i in range(n_star)],
}

with open("./data/star_meta.txt", "w") as f:
    json.dump(data, f)